## Start Up

In [412]:
import io
import os
import os.path
import re
import matplotlib.pyplot as plt
import cv2
import csv
import numpy as np
from PIL import Image
from io import BytesIO
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload
import torch
import torchvision

SCOPES = ['https://www.googleapis.com/auth/drive.file']
MIME_TYPE = 'application/vnd.google-apps.document'
APPLICATION_NAME = 'ipa-google-drive-api-client'

In [413]:
cd "C:\Users\covid\text_recognition"

C:\Users\covid\text_recognition


In [414]:
def get_service():

    # credentialの取得
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'google-drive-api.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # serviceの取得
    service = build('drive', 'v3', credentials=creds) 
    
    return service

def read_ocr(service, input_file, lang='jp'):
    # ファイルのアップロード

    # ローカルファイルの定義
    media_body = MediaFileUpload(input_file, mimetype=MIME_TYPE, resumable=True)

    # Google Drive上のファイル名
    newfile = 'output.pdf'

    body = {
        'name': newfile,
        'mimeType': MIME_TYPE
    }

    # 　creat関数でファイルアップロード実行
    # 同時にOCR読み取りも行う
    output = service.files().create(
        body=body,
        media_body=media_body,
        # ここで読み込み先言語の指定を行う
        ocrLanguage=lang,
    ).execute()

    # テキストファイルのダウンロード

    # リクエストオブジェクト生成
    request = service.files().export_media(
        fileId=output['id'],
        mimeType="text/plain"
    )
    output_path = 'output.txt'

    with open(output_path, 'a') as f:
        fh = io.FileIO(output_path, "wb")
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        service.files().delete(fileId=output['id']).execute()
    
        # テキストの取得
    with open(output_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 読み取り結果のリストを返す
    return lines[1:]


service = get_service()

In [415]:
def process_text_file(text_file):
    output_dir = "C:/Users/covid/text_recognition/output"
    if os.path.exists(output_dir):
        file_list = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
        for file_name in file_list:
            file_path = os.path.join(output_dir, file_name)
            os.remove(file_path)
    else:
        os.makedirs(output_dir)

    with open(text_file, 'r') as file:
        lines = file.readlines()

        lines = sorted(lines, key=lambda line: float(line.split()[1]))

        for i, line in enumerate(lines):
            line = line.strip()
            values = line.split()

            if len(values) == 5:
                object_class = values[0]
                a = float(values[1])
                b = float(values[2])
                c = float(values[3])
                d = float(values[4])

                # Calculate coordinates and dimensions
                x_center = int(wid * a)
                y_center = int(hei * b)
                width = int(wid * c)
                height = int(hei * d)

                x_min = x_center - width // 2
                y_min = y_center - height // 2
                x_max = x_center + width // 2
                y_max = y_center + height // 2

                output_filename = os.path.join(output_dir, f'book{i+1}.jpg')
                index = 1
                while os.path.exists(output_filename):
                    output_filename = os.path.join(output_dir, f'book{i+1}_{index}.jpg')
                    index += 1

                # Crop and save the image
                cropped = image.crop((x_min, y_min, x_max, y_max))
                cropped.save(output_filename)


## Execution

In [431]:
#Variable list
# ディレクトリのパス
directory_path = "C:/Users/covid/text_recognition/yolov7/runs/detect/"
# 画像ファイルの相対パスを指定
image_relative_path = "input.png"
# テキストファイルの相対パスを指定
text_file_relative_path = "labels/input.txt"

out_path = 'C:/Users/covid/text_recognition/output'
output_file = "C:/Users/covid/text_recognition/output_results.txt"

In [432]:
#import time

#time.sleep(15)

In [433]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()
cv2.imwrite("C:/Users/covid/text_recognition/yolov7/input.png",frame)

cap.release()

In [434]:
cd "C:\Users\covid\text_recognition\yolov7"

C:\Users\covid\text_recognition\yolov7


In [435]:
# ディレクトリ内のサブディレクトリのリストを取得
subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

# サブディレクトリの中で一番新しいものを取得
newest_subdirectory = max(subdirectories, key=lambda d: os.path.getctime(os.path.join(directory_path, d)))

# 最新のサブディレクトリのパスを作成
newest_subdirectory_path = os.path.join(directory_path, newest_subdirectory)

In [436]:
!python detect.py --source C:/Users/covid/text_recognition/yolov7/input.png --weights yolov7-e6e.pt --conf 0.25 --img-size 1280 --device 0 --save-txt

Namespace(weights=['yolov7-e6e.pt'], source='C:/Users/covid/text_recognition/yolov7/input.png', img_size=1280, conf_thres=0.25, iou_thres=0.45, device='0', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

15 books, Done. (41.0ms) Inference, (50.0ms) NMS
 The image with the result is saved in: runs\detect\exp52\input.png
Done. (0.520s)


YOLOR  v0.1-126-g84932d7 torch 2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

Model Summary: 792 layers, 151687420 parameters, 817020 gradients
C:\Users\covid\anaconda3\envs\localGPU\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [437]:
# 新しいディレクトリに移動
os.chdir(newest_subdirectory_path)

# 画像ファイルの絶対パスを作成
image_absolute_path = os.path.join(newest_subdirectory_path, image_relative_path)
# テキストファイルの絶対パスを作成
text_file_absolute_path = os.path.join(newest_subdirectory_path, text_file_relative_path)

In [438]:
# 画像をImageクラスのインスタンスに読み込む
image = Image.open(image_absolute_path)
# テキストファイルを読み込む
with open(text_file_absolute_path, 'r') as file:
    text_content = file.read()

In [439]:
# Process the text file
wid,hei = image.size
process_text_file(text_file_absolute_path)

PermissionError: [WinError 32] プロセスはファイルにアクセスできません。別のプロセスが使用中です。: 'C:/Users/covid/text_recognition/output\\book7.jpg'

In [ ]:
#output corresponding to list format
if __name__ == '__main__':
    output_list = []

    file_list = [filename for filename in os.listdir(out_path) if filename.endswith('.jpg')]
    file_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

    for filename in file_list:
        input_file = os.path.join(out_path, filename)
        output = read_ocr(service, input_file, 'ja')

        # 不要な文字（スペースとバックスラッシュ）を除去して一つの文字列に結合する
        cleaned_output = ''.join(line.strip().replace(' ', '').replace('/', '').replace('\n', '').replace('\\', '') for line in output)

        # 結果をリストに追加
        output_list.append(cleaned_output)

In [ ]:
# Save the results to the output file
with open(output_file, 'w', encoding='utf-8') as file:
        for result in output_list:
            file.write(result + '\n')

print(f"Results saved to {output_file}")

## Compare Methods of NLP

### Different.SequenceMatcher

In [ ]:
cd "C:\\Users\\covid\\text_recognition"

In [ ]:
import difflib

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    for index_a, text_a in enumerate(lines_a):
        max_similarity = 0.0
        best_match = None
        best_match_text_b = None

        for index_b, text_b in enumerate(lines_b):
            similarity = difflib.SequenceMatcher(None, text_a, text_b).ratio()

            if similarity > max_similarity:
                max_similarity = similarity
                best_match = text_b
                best_match_text_b = text_b

        print(f"Text A (line {index_a + 1}): {text_a.strip()}")
        print(f"Best Match in Text B (line {lines_b.index(best_match) + 1}): {best_match.strip()}")
        print(f"Highest Similarity Ratio: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()


In [ ]:
def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"
    output_file = "output_similarity.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    with open(output_file, "w", encoding="utf-8") as output:
        for index_a, text_a in enumerate(lines_a):
            for index_b, text_b in enumerate(lines_b):
                similarity = difflib.SequenceMatcher(None, text_a, text_b).ratio()

                output.write(f"Text A (line {index_a + 1}): {text_a.strip()}\n")
                output.write(f"Text B (line {index_b + 1}): {text_b.strip()}\n")
                output.write(f"Similarity Ratio: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


### Levenshtein distance

In [ ]:
import Levenshtein

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    for index_a, text_a in enumerate(lines_a):
        max_similarity = 0.0
        best_match = None

        for index_b, text_b in enumerate(lines_b):
            similarity = Levenshtein.ratio(text_a.strip(), text_b.strip())

            if similarity > max_similarity:
                max_similarity = similarity
                best_match = text_b

        print(f"Original Text A (line {index_a + 1}): {text_a.strip()}")
        print(f"Best Match for Text A (line {index_a + 1}): {best_match.strip()}")
        print(f"Highest Similarity Ratio: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()


In [ ]:
def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"
    output_file = "levenshtein_similarity.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    with open(output_file, "w", encoding="utf-8") as output:
        for index_a, text_a in enumerate(lines_a):
            max_similarity = 0.0
            best_match = None

            for index_b, text_b in enumerate(lines_b):
                similarity = Levenshtein.ratio(text_a.strip(), text_b.strip())

                output.write(f"Text A (line {index_a + 1}): {text_a.strip()}\n")
                output.write(f"Text B (line {index_b + 1}): {text_b.strip()}\n")
                output.write(f"Similarity Ratio: {similarity:.4f}\n\n")

                if similarity > max_similarity:
                    max_similarity = similarity
                    best_match = text_b

            output.write(f"Best Match for Text A (line {index_a + 1}): {best_match.strip()}\n")
            output.write(f"Highest Similarity Ratio: {max_similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


### Jaccard

In [ ]:
def jaccard_coefficient(s1, s2):
    set1 = set(s1)
    set2 = set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0.0

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    for index_a, text_a in enumerate(lines_a):
        max_similarity = 0.0
        best_match = None

        for index_b, text_b in enumerate(lines_b):
            similarity = jaccard_coefficient(text_a.strip(), text_b.strip())

            if similarity > max_similarity:
                max_similarity = similarity
                best_match = text_b

        print(f"Original Text A (line {index_a + 1}): {text_a.strip()}")
        print(f"Best Match for Text A (line {index_a + 1}): {best_match.strip()}")
        print(f"Highest Jaccard Coefficient: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()


In [ ]:
def jaccard_coefficient(s1, s2):
    set1 = set(s1)
    set2 = set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0.0

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"
    output_file = "jaccard_similarity.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    with open(output_file, "w", encoding="utf-8") as output:
        for index_a, text_a in enumerate(lines_a):
            for index_b, text_b in enumerate(lines_b):
                similarity = jaccard_coefficient(text_a.strip(), text_b.strip())

                output.write(f"Text A (line {index_a + 1}): {text_a.strip()}\n")
                output.write(f"Text B (line {index_b + 1}): {text_b.strip()}\n")
                output.write(f"Jaccard Coefficient: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    cosine_sim = calculate_cosine_similarity(lines_a + lines_b)

    output_file = "cosine_similarity_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            for j, line_b in enumerate(lines_b):
                similarity = cosine_sim[i][j]
                output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
                output.write(f"Text B (line {j + 1}): {line_b.strip()}\n")
                output.write(f"Cosine Similarity: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


In [ ]:
#全文出力
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_ngram_similarity(texts):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    cosine_sim = calculate_ngram_similarity(lines_a + lines_b)

    output_file = "ngram_similarity_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            for j, similarity in enumerate(cosine_sim[i][len(lines_a):]):
                output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
                output.write(f"Text B (line {j + 1}): {lines_b[j].strip()}\n")
                output.write(f"N-gram Similarity: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


In [ ]:
#最大値出力
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_ngram_similarity(texts):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    cosine_sim = calculate_ngram_similarity(lines_a + lines_b)

    output_file = "ngram_similarity_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            max_similarity = 0.0
            best_match = None
            best_match_index = -1
            for j, similarity in enumerate(cosine_sim[i][len(lines_a):]):
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_match = lines_b[j].strip()
                    best_match_index = j
            output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
            output.write(f"Best Match for Text A: {best_match}\n")
            output.write(f"Highest N-gram Similarity: {max_similarity:.4f}\n\n")
            if best_match_index != -1:
                print(f"Text A (line {i + 1}): {line_a.strip()}")
                print(f"Best Match for Text A: {lines_b[best_match_index].strip()}")
                print(f"Highest N-gram Similarity: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_partial_similarity(texts):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    cosine_sim = calculate_partial_similarity(lines_a + lines_b)

    output_file = "partial_similarity_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            for j, similarity in enumerate(cosine_sim[i][len(lines_a):]):
                output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
                output.write(f"Text B (line {j + 1}): {lines_b[j].strip()}\n")
                output.write(f"Partial Similarity: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_partial_similarity(texts):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    cosine_sim = calculate_partial_similarity(lines_a + lines_b)

    output_file = "partial_similarity_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            max_similarity = 0.0
            best_match = None
            best_match_index = -1
            for j, similarity in enumerate(cosine_sim[i][len(lines_a):]):
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_match = lines_b[j].strip()
                    best_match_index = j
            output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
            output.write(f"Best Match for Text A: {best_match}\n")
            output.write(f"Highest Partial Similarity: {max_similarity:.4f}\n\n")
            if best_match_index != -1:
                print(f"Text A (line {i + 1}): {line_a.strip()}")
                print(f"Best Match for Text A: {lines_b[best_match_index].strip()}")
                print(f"Highest Partial Similarity: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()


In [ ]:
import Levenshtein

def calculate_levenshtein_distance(texts):
    distance_matrix = [[0 for _ in range(len(texts[1]))] for _ in range(len(texts[0]))]
    for i in range(len(texts[0])):
        for j in range(len(texts[1])):
            distance_matrix[i][j] = Levenshtein.distance(texts[0][i], texts[1][j])
    return distance_matrix

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    levenshtein_distances = calculate_levenshtein_distance([lines_a, lines_b])

    output_file = "levenshtein_distance_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            for j, distance in enumerate(levenshtein_distances[i]):
                similarity = 1 - (distance / max(len(lines_a[i]), len(lines_b[j])))
                output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
                output.write(f"Text B (line {j + 1}): {lines_b[j].strip()}\n")
                output.write(f"Levenshtein Similarity: {similarity:.4f}\n\n")

if __name__ == "__main__":
    main()


In [ ]:
import Levenshtein

def calculate_levenshtein_distance(texts):
    distance_matrix = [[0 for _ in range(len(texts[1]))] for _ in range(len(texts[0]))]
    for i in range(len(texts[0])):
        for j in range(len(texts[1])):
            distance_matrix[i][j] = Levenshtein.distance(texts[0][i], texts[1][j])
    return distance_matrix

def main():
    file_a_path = "output_results.txt"
    file_b_path = "database.txt"

    with open(file_a_path, "r", encoding="utf-8") as file_a:
        lines_a = file_a.readlines()

    with open(file_b_path, "r", encoding="utf-8") as file_b:
        lines_b = file_b.readlines()

    levenshtein_distances = calculate_levenshtein_distance([lines_a, lines_b])

    output_file = "levenshtein_distance_results.txt"
    with open(output_file, "w", encoding="utf-8") as output:
        for i, line_a in enumerate(lines_a):
            max_similarity = 0.0
            best_match = None
            best_match_index = -1
            for j, distance in enumerate(levenshtein_distances[i]):
                similarity = 1 - (distance / max(len(lines_a[i]), len(lines_b[j])))
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_match = lines_b[j].strip()
                    best_match_index = j
            output.write(f"Text A (line {i + 1}): {line_a.strip()}\n")
            output.write(f"Best Match for Text A: {best_match}\n")
            output.write(f"Highest Levenshtein Similarity: {max_similarity:.4f}\n\n")
            if best_match_index != -1:
                print(f"Text A (line {i + 1}): {line_a.strip()}")
                print(f"Best Match for Text A: {lines_b[best_match_index].strip()}")
                print(f"Highest Levenshtein Similarity: {max_similarity:.4f}\n")

if __name__ == "__main__":
    main()
